Your solution should have the following steps:

1- Exploratory data analysis: check columns to understand datatypes, value ranges, find possible missing values, the correlation between features, and create relevant data visualization

2- Linear learner: create an ML pipeline with a linear model. Explicitly mention your model assumptions and what you did to make sure those assumptions hold

3- non-linear model: also fit a non-linear model of your choice to compare the result

4- ensemble model: use any of the discussed ensemble methods to create a new model based on step 2 and 3

5- Use the model from step 4 to create a fucntion called credit_approval that takes a single row of data and predict if a credit card will be granted or not. It should looklike something like below.

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
!conda install --yes scikit-learn

Solving environment: done

## Package Plan ##

  environment location: /Users/nahee/opt/anaconda3/envs/pratt_ml

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2021.10.8          |   py38hecd8cb5_0         151 KB
    ------------------------------------------------------------
                                           Total:         151 KB

The following packages will be UPDATED:

  certifi                          2021.5.30-py38hecd8cb5_0 --> 2021.10.8-py38hecd8cb5_0



certifi-2021.10.8    | 151 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
!conda install --yes seaborn

Solving environment: done

# All requested packages already installed.



In [5]:
import sklearn

In [6]:
#Import data

df = pd.read_csv('data/crx.data', header=None, names=["A1","A2","A3","A4","A5","A6","A7","A8","A9","A10","A11","A12","A13","A14","A15","A16"])
print(df.head())
df

  A1     A2     A3 A4 A5 A6 A7    A8 A9 A10  A11 A12 A13    A14  A15 A16
0  b  30.83  0.000  u  g  w  v  1.25  t   t    1   f   g  00202    0   +
1  a  58.67  4.460  u  g  q  h  3.04  t   t    6   f   g  00043  560   +
2  a  24.50  0.500  u  g  q  h  1.50  t   f    0   f   g  00280  824   +
3  b  27.83  1.540  u  g  w  v  3.75  t   t    5   t   g  00100    3   +
4  b  20.17  5.625  u  g  w  v  1.71  t   f    0   f   s  00120    0   +


,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,00200,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-


# 1. Exploratory data analysis
## 1-1 Understand datatypes

In [7]:
#data type
print(df.info())

#total 690 entries
#3 types of data: float64, int64, object.
#float64, int64 is numeric data, object is non-numeric data
#--> numeric data: A3, A8, A11, A15 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      690 non-null    object 
 1   A2      690 non-null    object 
 2   A3      690 non-null    float64
 3   A4      690 non-null    object 
 4   A5      690 non-null    object 
 5   A6      690 non-null    object 
 6   A7      690 non-null    object 
 7   A8      690 non-null    float64
 8   A9      690 non-null    object 
 9   A10     690 non-null    object 
 10  A11     690 non-null    int64  
 11  A12     690 non-null    object 
 12  A13     690 non-null    object 
 13  A14     690 non-null    object 
 14  A15     690 non-null    int64  
 15  A16     690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB
None


## 1-2 Value ranges

In [8]:
#stats of numeric data
print(df.describe())
#found out min/max value ranges on numeric data

               A3          A8        A11            A15
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


## 1-3 Find possible missing values

In [22]:
#find missing value
df.isnull().values.sum()

0

In [23]:
#find missing value2
#Change '?' to nan
df = df.replace('?', np.nan)

#find 'nan' value
print(str(df.isnull().values.sum()))
print(df.isnull().sum())
#--> Total 67 missing values

67
A1     12
A2     12
A3      0
A4      6
A5      6
A6      9
A7      9
A8      0
A9      0
A10     0
A11     0
A12     0
A13     0
A14    13
A15     0
A16     0
dtype: int64


## 1-4 Correlation between features

In [24]:
corrMatrix = df.corr()
print (corrMatrix)

           A3        A8       A11       A15
A3   1.000000  0.298902  0.271207  0.123121
A8   0.298902  1.000000  0.322330  0.051345
A11  0.271207  0.322330  1.000000  0.063692
A15  0.123121  0.051345  0.063692  1.000000


## 1-5 Create relevant data visualization

In [52]:

sns.relplot(x="", y="", data=corrMatrix)

ValueError: Could not interpret value `` for parameter `x`